In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_info = df['Info'].tolist()

print('总数量：' + str(len(list_info)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for info in list_info:
    work.put_nowait(info)

def crawler():
    global df
    
    while not work.empty():
        info = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(info,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_tr = html.xpath('//table[contains(@class, "moreinfotable")]/tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Info': info}])
                
                for tr in list_tr[1:]:
                    df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip().lower()] = tr.xpath('./td[2]/text()')[0].strip()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Info': info}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(info +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./specification_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：17

https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=9719692  <->  [ok] - 剩余数量：12
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=3786267  <->  [ok] - 剩余数量：11
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=12769041  <->  [ok] - 剩余数量：10
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=13953849  <->  [ok] - 剩余数量：9
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=13953853  <->  [ok] - 剩余数量：8
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=15195333  <->  [ok] - 剩余数量：7
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=15195341  <->  [ok] - 剩余数量：6
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=15195345  <->  [ok] - 剩余数量：5
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=13953805  <->  [ok] - 剩余数量：4
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=15195329  <->  [ok] - 剩余数量：3
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=15195365  <->  [ok] - 剩余数量：2
https://www.rockauto.com/en/moreinfo.php?pt=5061&pk=15195357  <->  [ok] - 剩余数量：1
https://www.rockaut